# An Agent Based Classroom Seating Model

In [ ]:
''' Set up this document and import libraries '''
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
%matplotlib notebook
rcParams['figure.figsize'] = (6, 4)

from ipywidgets import interact, interactive
import ipywidgets as widgets

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

## Introduction

In this notebook, we aim to demonstrate some of the features of our model in an interactive manner. Feel free to change some of the values to see the resulting outcome.

Full details on the model's stucture and behaviour are described in our paper *An Agent-Based Model of Classroom Seating Behaviour*. This document only outlines how to use our `Python` implementation. The code, along with the paper and this document can be found at: https://github.com/WavyV/ABM_RocketMan

## Setting up the Environment

To begin, we must define the characteristics of the model's environment, that is the layout of the classroom. This is setup with two main variables:

* The number of rows, `num_rows`
* A list of blocks sizes, `blocks`, where each element describes the width of a seating section, which are separated by verticle aisles. Can be 0.
 
Note the number of rows also counts for any horizontal aisles, which there is asssumed to be one at the front of the classroom. This can be specfied in a third variable `aisles_y`. This however does not affect the agents' movement, but it is merely an asthetic feature when plotting.

In [ ]:
''' Define the Classroom shape '''

num_rows = 14
blocks = [6, 14, 0]
aisles_y = [0]

Next, it possible to score the seats in the classroom a desireability value, from the range [0, 1]. That is, seats with a score 0 are avoided by the students (e.g. the front rows seats), and seats with a score 1 are highly sought after (e.g. the seats in the middle that have a good view). These values are stored in a `numpy` array named `pos_utilities`. The shape of the array does not necessarily have to include the aisles (the model can fill these in). This should be indexed as (Column, Row) since we are dealing with (x, y) positional data. Also note the front row of the class is at index 0.

This array can be generated in a varienty of ways, such as with some 2D distribution, but here we will keep it simple and split the classroom into sections and manually define some scores loosly based on some observed data.

In [ ]:
''' Define the positional utilities of the classroom '''

pos_utilities = np.ones((sum(blocks), num_rows-1))

# use the blocks to calculate the aisles' locations 
# and use them to define sections
aisles = np.cumsum(blocks)

# first three rows very undesirable
pos_utilities[aisles[0]+1:, 0:3] *= 0.05
pos_utilities[0:aisles[0]+1, 0:3] *= 0.0

# back five rows somewhat desirable
pos_utilities[aisles[0]+1:, -5:] *= 0.5
pos_utilities[0:aisles[0]+1, -5:] *= 0.2

# middle section to the left also quite desirable
pos_utilities[0:aisles[0]+1, 3:-5] *= 0.7

''' Plot this utility map '''
plt.figure()
plt.imshow(pos_utilities.T, cmap='gray')
plt.title('Positional Desireabilities of the Classroom')
plt.colorbar()
plt.show()

Finally, these parameters are stored in an instance of the `ClassroomDesign` class, for reuse and referencing later.

In [ ]:
from model import ClassroomDesign

''' Create the classroom object '''
classroom = ClassroomDesign(blocks=blocks,
                            num_rows=num_rows,
                            aisles_y=aisles_y,
                            pos_utilities=pos_utilities)

## The Agents

The students in the simulation have numerous perameters that facilitate dynamic behaviour. 

First, each student has a *sociability* factor that determines how comfortable they are sitting next to someone that they do not know. Negative values indicate social aversion (e.g. rather sit alone), zero represents indiffernece, and positive values for a sociable student, willing to meet new people. (The exact range of these values does not matter, since they will be normalised in the utility calulations).

To set this in the model, we must generate a list of sociabilities to use, one for each student. This can be generated in a few ways, e.g. sampled from a suitable probability distribution (such as Uniform, Normal, Cauchy etc). For this example however, we will instead draw from a distribution generated from some real data collected on this behaviour.

In [ ]:
from data_processing.process_form import agent_sociability_gen

''' Set the students sociabilities '''
class_size = 150
sociability_generator = agent_sociability_gen()
sociability_sequence = [next(sociability_generator) for _ in range(class_size)]

plt.figure()
plt.hist(sociability_sequence)
plt.title('Distribution of Sociability Parameters')
plt.show()

Next, we must setup the *social network* of the class. Students are likely to sit next to other friends in the room, provided it is not too difficult to reach them. This network is a unweighted, undirected graph where each edge indicates a mutual friendship, and this is stored as `social_network`, an NxN matrix. Students are likely to sit next to other friends in the room, provided it is not too difficult to reach them.

Again, there are a variety of possibilities to generate this social graph for experiments. Just like the sociabilities, we can use real data that we also collected. However, for simplicity we will use the library `networkx` to create an Erdős-Renyi graph, which has been shown to accurately model real networks.

In [ ]:
import networkx as nx

''' Set the social network '''
er_graph = nx.erdos_renyi_graph(class_size, 0.2)
social_network = nx.to_numpy_matrix(er_graph)

#  To use the real data, uncomment the following:

# from data_processing.process_form import agent_friends_gen
# from social.network import walts_graph
# friendship_generator = agent_friends_gen()
# degree_sequence = [next(friendship_generator) for _ in range(class_size)]
# social_network = walts_graph(degree_sequence, plot=False)[0]

## Model Setup

The model we have created is essentially a discrete choice model, and so a crucial component of the model is the *utility function*. The utility of a seat $\vec{x}_i$ for student $i$ takes the form:

$$
\begin{equation}
u_i(\vec{x}_i) = \beta_1 f_i + \beta_2 s_i + \beta_3 p_i + \beta_4 a_i
\end{equation}
$$

(A full description of how this utility is calculated can be found in the paper). For a simulation run, we must define the coefficients of this function, namely the $\beta$s. This is stored as an ordered list `coeffs`. Note that while the model requires that each $\beta$ is in the range (0, 1) and they must total 1, the model will automatically normalise any set of coefficients to meet these requirements. As expected, the values of these coefficients have a dramatic impact on the model's dynamics, and is the center of the study.

In [ ]:
''' Define the utility coefficients '''

coefs = [
    0.7,     # positional utility
    1,   # friendship utility
    0,     # sociability 
    1      # accessability
]

Finally, there are two modes in which to run the model, namely *deterministic choice* (where student make a fully rational choice and pick the highest utility seat) or *stochastic choice* (where the utilities are interpreted as a probability). The latter mode arguably produces more realistic results. The mode can be set with the boolean `deterministic_choice`.

Now that all the components have been set up, we can finally initialise the model.

In [ ]:
from model import ClassroomModel

''' Initialise the model '''

classroom_model = ClassroomModel(classroom_design=classroom,
                                 coefs=coefs,
                                 social_network=social_network,
                                 seed=1,
                                 deterministic_choice=False)

We can observe the model as it is now with the model's built in `plot()` function. The front of the classroom is the top of the image. The aisles are clearly marked (light orange colour) and it can been seen the model has added doors to the plot (dark red, these are only asthetic). Hovering the mouse over the plot will reveal information about that particular cell. Later, as students enter the room, information can also be seen on their status.

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots()
classroom_model.plot(fig, ax, state=0, interactive=True)

## Running the Model

To generate the next step of the model, simply call `classroom_model.step()`. This will add a new student to the room and update the current seating positions and model state. This is repeated until all students have taken a seat.

In [ ]:
''' Run the model to generate data '''

for i in range(class_size):
    classroom_model.step()

Every state of the model is stored within the object for later analysis. We can plot the model states as before, and using a simple widget setup you can interactively scroll through each step and observe the simulation's growth. (If the plot is non-responsive, try re-running the cell).

In [ ]:
%matplotlib notebook

''' Create an interactive plot '''

fig, ax = plt.subplots()
classroom_model.plot(fig, ax, state=0, interactive=True)

def scroll(s):
    classroom_model.plot(fig, ax, state=s['new'], interactive=True)

play = widgets.Play(interval=1000, value=0, min=0, max=class_size, step=1, disabled=False)
slider = widgets.IntSlider(min=0, max=class_size, step=1, continuous_update=False)
slider.observe(scroll, 'value')
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.Box([play, slider])

In the plot, students are represented by the green squares, with the darker shades representing 'happier' students (their updated utility). Hover the mouse over a student reveals their status and infrmation about the seat they have selected.

After observing the above animation, it is easy to try different parameters in the model setup and watch how the model responds (simply re-run all the cells after making any changes). For example, you can explore different values that may lead to a more diverse scattering of the students, or clustering behaviour. It is also easy to explore the impact alternate social networks have, for example a simulation where no one knows each other (i.e. start of a new year).

## Analysis

With a simulation complete, it is possible to run some analysis of the final distribution. For simplicity we will only look at the final seating positions, however futher analysis can be made on the time-series growth of the model. In demonstrating these methods we will be comparing the model from above with a different simulation, one that is almost completely random.

In [ ]:
''' Create a comparison model '''

new_coefs = [0, 0, 0, 0]
random_model = ClassroomModel(classroom_design=classroom, coefs=new_coefs,
                              social_network=social_network, seed=1)

for i in range(class_size):
    random_model.step()
    
fig, ax = plt.subplots()
random_model.plot(fig, ax, state=-1, interactive=False)

The file `model_comparison.py` contains a few simple metrics for analysing a model state. Since we are only dealing with final positions and no other data, we can use the method `model.get_binary_model_state()` which returns a binary matrix with ones representing taken seats and zeros otherwise (aisles are also striped from this matrix). 

The most basic metric is the clustering distribution, that is the total counts of all the various lengths of students sat in an unbroken row. The histogram can be produced as below.

In [ ]:
from model_comparison import count_clusters

plt.figure()
plt.plot(count_clusters(classroom_model.get_binary_model_state()), label='classroom_model')
plt.plot(count_clusters(random_model.get_binary_model_state()), label='random_model')
plt.legend()
plt.show()

As can be observed, the random simulation yields a large number of smaller group sizes, which suggests a wide, sparse distribution of students, whereas the other model appears to have formed a range of cluster sizes, indicating a tighter density. However, this measure is limited in its ability to capture information regarding the overall structure, since it is invariant to row ordering.

Another suitable metric is the 2D entropy of the distribution, which gives information about the structure for a range of levels, from the local neighbourhood of each student to the full classroom.

In [ ]:
from model_comparison import get_entropy

plt.figure()
plt.plot(get_entropy(classroom_model.get_binary_model_state()), label='classroom_model')
plt.plot(get_entropy(random_model.get_binary_model_state()), label='random_model')
plt.legend()
plt.show()

This graph suggests a well defined structure for the first model on all scales compared to the 'random' model. The usefulness of this metric is it's invariance to rotation and mirroring, and can compare differently shaped classrooms.

Metrics such as these allow us to quantify and compare different simulations to descern the contribution each parameter has on the model. These tools and others are used in the sensitivity analysis, validation and parameter estimation of our paper.